# ДЗ №2 Оценка качества Visual Language Model в задачах e-com

Вы работаете в R&D-подразделении крупного e-commerce-сервиса.  
В компании уже используются разные модели для:
- классификации товаров по категориям,
- генерации описаний и заголовков,
- извлечения признаков для поиска и рекомендаций.

Хочется заменить этот зоопарк моделей на **единую мультимодальную модель (VLM)**,  
которая могла бы решать несколько задач сразу.

Ваша задача — построить **мини-бенчмарки** для оценки мультимодальных моделей  на реальном датасете товаров, провести сравнение моделей и сделать вывод,  
какая модель лучше подходит для внедрения в e-commerce-сценарий.


## Исходные данные

В рамках задания предлагается использовать датасет [**AMAZON-Products-2023**](https://huggingface.co/datasets/milistu/AMAZON-Products-2023),  
содержащий данные о товарах (изображения, названия, описания, категории, цены и пр.).  

Размер датасета — более 100 000 объектов,  
поэтому вам необходимо выбрать **подмножество 1000–2000 объектов** для экспериментов. Какие именно объекты выбирать - необходимо определить Вам, чтобы бенчмарки были как можно более репрезентативными.

---

## Часть 1. Классификация / регрессия

1. Сформулируйте задачу, которую можно оценить классическими метриками  
   (например, классификация категории товара, предсказание диапазона цены, определение бренда и т.д.).

2. Подготовьте данные (изображения, названия, метки) в удобном для инференса формате.

3. Примените 2–3 мультимодальные модели (например, Qwen2.5-VL, LLaVA-OneVision, InternVL). Обратите внимание на время инференса каждой модели, оцените необходимые ресурсы заранее, чтобы не упереться по времени в дедлайн.

4. Рассчитайте **эвристики и метрики** для оценки качества модели:
   - accuracy, F1, MSE, cosine similarity, и др.
   - опишите, почему вы выбрали именно эти метрики.

5. Сделайте **анализ результатов**:
   - где модели ошибаются;
   - какие категории / признаки наиболее сложны;
   - какие факторы влияют на качество.

---

## Часть 2. Генерация названий и описаний товаров

1. Поставьте задачу генерации:
   - сгенерировать **название** по изображению/описанию;
   - сгенерировать **описание** по изображению/категории.

2. Сравните результаты моделей:
   - автоматические метрики: BLEU, ROUGE, CLIPTextSim;
   - использование **LLM-as-a-Judge** — примените языковую модель как «судью»,  
     которая сравнивает сгенерированный и эталонный текст. Вы можете самостоятельно выбрать данную модель.

3. Оцените, какие модели лучше справляются с задачей описания,  
   а какие — с генерацией коротких названий, как влияет использование разных модальностей на результат.

---

## Оценка и фреймворки

- Основная часть — ваша собственная реализация метрик и LLM-Judge.  
- **Дополнительные баллы:**  
  если вы интегрируете один из существующих фреймворков для оценки качества:
  - [`LMMs-Eval`](https://github.com/EvolvingLMMs-Lab/lmms-eval)  
  - или другой open-source тулкит для мультимодальной оценки.

---

## Формат работы

1. Выполняйте шаги в ноутбуке. Если часть реализована в отдельных скриптах или нескольких ноутбуках - загрузите в качестве решения все артефакты.
2. Все визуализации (графики, таблицы, примеры товаров) должны быть в ноутбуке.  
3. Каждый блок должен содержать краткое пояснение в Markdown.  
4. Код должен быть воспроизводимым (укажите seed, версии библиотек и используемые модели).

---

## Альтернативный вариант (по согласованию с ведущим преподавателем)

Если вы хотите использовать **другой датасет**, согласуйте его с преподавателем.  
Требования к альтернативному датасету:
- 1000–2000 объектов в бенчмарках;
- наличие визуальной и текстовой информации;
- возможность поставить **две разные задачи**:
  1. одна решается с помощью метрик (классификация/регрессия),
  2. другая оценивается через **LLM-as-a-Judge** или CLIPScore-подобные метрики.

---

## Критерии оценивания

| Компонент | Баллы |
|------------|-------|
| Реализация и анализ классификации / регрессии | 2 |
| Генерация и оценка текстов (Без использования LLM) | 2 |
| Генерация и оценка текстов (LLM-as-Judge) | 2 |
| Использование фреймворков (LMMs-Eval, etc) | по 1 баллу за каждый отдельный бенчмарк |
| Анализ, визуализация, качество отчёта | 2 |

Максимум: 10 баллов

---


При необходимости, можно поднять собственный LLM-сервер в Colab, чтобы можно было использовать его для оценки качества и инференса с локальной машины.

## Установка зависимостей

In [ ]:
!pip install vllm pyngrok fastapi uvicorn --quiet



### Что такое ngrok
**ngrok** — это инструмент, который позволяет создать **временный публичный доступ (туннель)**  
к вашему локальному серверу, запущенному внутри Colab или на вашем компьютере.

Colab-ноутбук работает в изолированной среде без внешнего IP-адреса.  
Если мы поднимаем внутри него API-сервер (например, модель с OpenAI-совместимым API),  
другие сервисы (или наши скрипты вне Colab) не смогут к нему подключиться напрямую.  

**ngrok** решает эту проблему:
- он “пробрасывает” локальный порт (например, `localhost:8000`) наружу,  
- создаёт временный HTTPS-URL (например, `https://abc123.ngrok.io`),  
- и перенаправляет запросы с этого URL внутрь Colab.

*Пример:*  
Вы запустили LLM-судью в Colab на порту `8000` → ngrok создаёт публичную ссылку,  
которую можно использовать в коде как `openai.api_base = "https://abc123.ngrok.io/v1"`.  
Теперь вашу LLM в Colab можно вызывать из внешних программ так же, как OpenAI API.


## Авторизация ngrok

Зарегистрируйтесь на https://dashboard.ngrok.com/get-started/your-authtoken и вставьте свой токен:

In [ ]:
from pyngrok import ngrok
NGROK_AUTH_TOKEN = "ваш_токен"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


Проброс наружу

In [ ]:
public_url = ngrok.connect(8000, "http")
print(f"Публичный URL API: {public_url}")

Вы увидите что-то вроде http://abcd1234.ngrok.io


### Что такое **vLLM**

**vLLM** — это оптимизированный серверный движок для **быстрого инференса LLM**  
(в том числе больших мультимодальных моделей вроде Qwen2.5-VL или LLaVA).
- Загружает любую модель из Hugging Face (`transformers`) и запускает её как API-endpoint.  
- Совместим с **OpenAI API-форматом**, т.е. можно обращаться к локальной модели через `openai.ChatCompletion.create(...)`.  
- Поддерживает batch-инференс, streaming, CUDA-ускорение и управление памятью (PagedAttention).  
- Может использоваться для нужного нам поднятия *локального LLM-судьи* прямо в Colab.

## Запуск vLLM сервера (OpenAI-совместимого)

In [ ]:
!nohup python3 -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen2-1.5B-Instruct \
  --host 0.0.0.0 \
  --port 8000 \
  > server.log 2>&1 &

Это займёт 1–2 минуты — vLLM подгрузит модель и запустит API. Смотрите на вывод в логах.

Проверка, что сервер запущен

In [ ]:
!curl http://127.0.0.1:8000/v1/models

Увидете что-то вроде:

`{"object":"list","data":[{"id":"Qwen/Qwen2-1.5B-Instruct","object":"model","created":1761208147,"owned_by":"vllm","root":"Qwen/Qwen2-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-ec699a8339664cee8b54aeaf323a7f4e","object":"model_permission","created":1761208147,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}`

## Вызов API извне

Теперь вы можете стучаться к серверу снаружи, как к OpenAI API:

In [ ]:
import requests

API_URL = "http://abcd1234.ngrok.io/v1/chat/completions"  #  замените URL
headers = {"Content-Type": "application/json"}
data = {
    "model": "Qwen/Qwen2-1.5B-Instruct",
    "messages": [
        {"role": "user", "content": "Привет! Расскажи шутку про ИИ."}
    ],
    "temperature": 0.7
}

r = requests.post(API_URL, headers=headers, json=data)
print(r.json()["choices"][0]["message"]["content"])


## Использование как OpenAI клиента

In [ ]:
!pip install openai --quiet

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://abcd1234.ngrok.io/v1",
    api_key="dummy"  # vLLM не проверяет ключ
)

response = client.chat.completions.create(
    model="Qwen/Qwen2-1.5B-Instruct",
    messages=[{"role": "user", "content": "Расскажи о мультимодальных моделях простыми словами."}]
)

print(response.choices[0].message.content)
